In [ ]:
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from util import prepare_df

In [ ]:
%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}
sns.set()
sns.set(rc={"figure.figsize": (11.7, 8.27)})

In [ ]:
df = prepare_df(xp_identifier="ac_hi_od_hi_idle")

In [ ]:
task_palette = sns.color_palette("deep", int(df["n_task"].unique().shape[0]))
sns.set_theme(style="whitegrid", palette="deep")

In [ ]:
scope = df["search_type"] == "BFS"
scope &= df["unsafe"].isna()
df_ref = df.loc[scope, ["tid", "duration_s", "n_visited"]].rename(
    columns={"duration_s": "duration_s_ref", "n_visited": "n_visited_ref"}
)
df = df.merge(df_ref, on="tid")

In [ ]:
df["n_visited_norm"] = df["n_visited"]
df["duration_s_norm"] = df["duration_s"]

df.loc[~scope, "n_visited_norm"] = (
    df.loc[~scope, "n_visited"] / df.loc[~scope, "n_visited_ref"] - 1
) * 100
df.loc[~scope, "duration_s_norm"] = (
    df.loc[~scope, "duration_s"] / df.loc[~scope, "duration_s_ref"] - 1
) * 100

In [ ]:
import pandas as pd

pd.set_option("display.float_format", lambda x: "%.4f" % x)

In [ ]:
df

In [ ]:
df["Search"] = df["search_type"]
df["Oracle"] = df["unsafe"].apply(
    lambda x: "HI over demand" if x == "hi_interference" else "None"
)

In [ ]:
dfagg = (
    df.groupby(["Search", "Oracle"], dropna=False)[
        ["n_visited_norm", "duration_s_norm"]
    ]
    .agg(["min", "mean", "std", "median", "max"])
    .apply(lambda x: round(x, 2))
    .applymap(str)
)

In [ ]:
dfagg.columns = dfagg.columns.set_levels(
    ["Number of visited states before halt", "Execution time"], level=0
)

In [ ]:
dfagg = dfagg.reindex(["BFS", "ACBFS"], level=0)
dfagg = dfagg.reindex(["None", "HI over demand"], level=1)

In [ ]:
dfagg.iloc[1:] = " (" + dfagg.iloc[1:] + "\%)"

In [ ]:
dfagg["Number of visited states before halt"].T

In [ ]:
print(dfagg["Number of visited states before halt"].T.to_latex())

In [ ]:
dfagg2_int = (
    df.groupby(["Search", "Oracle"], dropna=False)[["n_visited"]]
    .agg(["min", "mean", "std", "median", "max"])
    .applymap(lambda x: round(x))
)

dfagg2_int = dfagg2_int.reindex(["BFS", "ACBFS"], level=0)
dfagg2_int = dfagg2_int.reindex(["None", "HI over demand"], level=1)

dfagg2_int = dfagg2_int.T

dfagg2 = dfagg2_int.applymap(str)


dfagg2

In [ ]:
r = dfagg2_int.iloc[:, 0]
for i in range(3):
    dfagg2_int.iloc[:, 1 + i] /= r
    dfagg2_int.iloc[:, 1 + i] -= 1

In [ ]:
(dfagg2_int * 100).applymap(lambda x: round(x))